Exercise 4

In [2]:
from web3 import Web3
from matplotlib import pyplot as plt
import json
import requests
import pandas as pd

In [3]:
getblock_url= 'https://rsk.getblock.io/[YOUR API KEY]/mainnet/'
web3= Web3(Web3.HTTPProvider (getblock_url))
web3.isConnected()

False

In [ ]:
end_block =  web3.eth.getBlock(block_identifier='latest').number
start_block = end_block-100

transaction_count= []
for block in range(start_block, end_block):
    counts= (web3.eth.getBlockTransactionCount(block))
    transaction_count.append (counts)

In [ ]:
plt.plot (transaction_count)
plt.xlabel("100 latest blocks")
plt.ylabel("Number of transactions per block")
plt.show()

Exercise 5

In [ ]:
end_block =  web3.eth.getBlock(block_identifier='latest').number
start_block = end_block-100

recepients= []
for block in range(start_block, end_block):
    for tx_hash in (web3.eth.getBlock(block_identifier=block).transactions):
        to_address = web3.eth.get_transaction(tx_hash).to
        recepients.append (to_address)

In [ ]:
latestblocks_recepients = {i:recepients.count(i) for i in recepients}
latestblocks_recepients

Exercise 6

In [ ]:
infura_url= 'https://mainnet.infura.io/v3/[YOUR API KEY]'
web3= Web3(Web3.HTTPProvider (infura_url))
web3.isConnected()

In [ ]:
sc_address="0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D"
with open ("./ba_abi.json") as f:
    abi= json.load (f)
baContract = web3.eth.contract(address=sc_address, abi=abi)

In [ ]:
url="https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"
response= requests.get (url)
data= response.json()
ether_price= data ['ethereum']['usd']

In [ ]:
owners=[]
balances = []
ape_ids=[]
for ape_id in range (0,10000):
    ape_ids.append (ape_id)
    owner= baContract.functions.ownerOf(ape_id).call()
    owners.append (owner)
    balance = (web3.fromWei(web3.eth.getBalance(owner), "ether"))
    balances. append ((float (balance))* ether_price)

In [ ]:
dict_todf= {'Ape_ids':ape_ids,'Bayc_Owners':owners, 'Address_balance':balances}
df= pd.DataFrame (dict_todf, columns= ["Id","Bayc_Owners", "Address_balance"])
df.head()

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(subset=['Bayc_Owners'], inplace= True)
df.shape

In [ ]:
print ("The average in USD (at current price) of a BAYC holder is $",df.Address_balance.mean())